For this homework, you are required to download the data, try different methods, and show the performance (Root-Mean Squared Error, RMSE) of your best method on the probe set. As you already have the ratings for the probe set, be careful not to over-fit. Take it as a test set.

You are supposed to do in your project team, and submit one report per team.

Grading (0-100):

<b>Basics</b>: first working predictors, performance and diagnose, how to improve and limit overfitting, descriptions of predictors used, etc (60%)

<b>Clarify of your report</b>: in addition to the description of the process of improving your predictors, clearly state which member has accomplished which part(s). (20%)

<b>Performance</b>: based on your RMSE on the probe set and if necessary, on the qualifying set. (20%)

In [50]:
import numpy as np
import pandas as pd
import os
import platform
import pickle
import json

In [2]:
if platform.system() == 'Windows':
    !dir
else:
    !ls -la

total 4631937
drwxrwxr-- 1 vasia plugdev      4096 Nov 20 03:13 .
drwxrwxr-- 1 vasia plugdev      4096 Nov 19 15:20 ..
-rwxrwxr-- 1 vasia plugdev    119991 Nov 20 00:04 avatar.png
-rwxrwxr-- 2 vasia plugdev      3824 Nov 20 00:05 cleaner.ipynb
-rwxrwxr-- 2 vasia plugdev 495032008 Nov 19 15:20 combined_data_1.txt
-rwxrwxr-- 2 vasia plugdev 555207357 Nov 19 15:21 combined_data_2.txt
-rwxrwxr-- 2 vasia plugdev 465157779 Nov 19 15:21 combined_data_3.txt
-rwxrwxr-- 2 vasia plugdev 552537802 Nov 19 15:21 combined_data_4.txt
-rwxrwxr-- 1 vasia plugdev 609892706 Nov 19 22:40 combined_data_cleaned_1.txt
-rwxrwxr-- 1 vasia plugdev 690067046 Nov 19 22:41 combined_data_cleaned_2.txt
-rwxrwxr-- 1 vasia plugdev 596988794 Nov 19 22:41 combined_data_cleaned_3.txt
-rwxrwxr-- 1 vasia plugdev 713592119 Nov 19 22:42 combined_data_cleaned_4.txt
drwxrwxr-- 1 vasia plugdev      4096 Nov 19 21:41 .git
-rwxrwxr-- 2 vasia plugdev       147 Nov 19 17:29 .gitignore
drwxrwxr-- 1 vasia plugdev         0 Nov 19 16:3

In [62]:
movie_titles = pd.read_csv('movie_titles_cleaned.csv', skipinitialspace=True,
                           header=None,
                           names=["movie_id","year","title"],
                           dtype={"movie_id":np.int32,"year":np.float32,"title":str},
                           encoding='ISO-8859-1'
                          ).fillna('')
                        


In [63]:
movie_titles[:10]

,movie_id,year,title
0,1,2003,Dinosaur Planet
1,2,2004,Isle of Man TT 2004 Review
2,3,1997,Character
3,4,1994,Paula Abdul's Get Up & Dance
4,5,2004,The Rise and Fall of ECW
5,6,1997,Sick
6,7,1992,8 Man
7,8,2004,What the #$*! Do We Know!?
8,9,1991,Class of Nuke 'Em High 2
9,10,2001,Fighter


In [6]:
# data = pd.read_csv('combined_data_cleaned_1.txt',
#                    header=None,
#                    names=['movie_id','customer_id','rating','date'],
#                    infer_datetime_format=True,
#                    parse_dates=True,
#                    dtype={'movie_id':np.int32,'customer_id':np.int32,
#                            'rating':np.int32}
#                   )

data = pd.concat((pd.read_csv('combined_data_cleaned_' + str(i) + '.txt',
                              header=None,
                              names=['movie_id','customer_id','rating','date'],
                              infer_datetime_format=True,
                              parse_dates=True,
                              dtype={'movie_id':np.int32,'customer_id':np.int32,
                                     'rating':np.int32},
                              low_memory=True
                             ) 
                  for i in range(1,5)))

In [13]:
data[:10]

,movie_id,customer_id,rating,date
0,1,1488844,3,2005-09-06
1,1,822109,5,2005-05-13
2,1,885013,4,2005-10-19
3,1,30878,4,2005-12-26
4,1,823519,3,2004-05-03
5,1,893988,3,2005-11-17
6,1,124105,4,2004-08-05
7,1,1248029,3,2004-04-22
8,1,1842128,4,2004-05-09
9,1,2238063,3,2005-05-11


In [27]:
### this assertion passes. Our id's range from 1 to 17770 inclusive.
# assert(list(data['movie_id'].unique()) == list(range(1,17771)))

### Calculate average rating for each movie
average_ratings = {i: data[data['movie_id'] == i]['rating'].mean() for i in range(1,17771)}





In [28]:
average_ratings

{1: 3.749542961608775,
 2: 3.5586206896551724,
 3: 3.6411530815109345,
 4: 2.73943661971831,
 5: 3.9192982456140353,
 6: 3.084396467124632,
 7: 2.129032258064516,
 8: 3.1898054996646548,
 9: 2.6210526315789475,
 10: 3.180722891566265,
 11: 3.0303030303030303,
 12: 3.4175824175824174,
 13: 4.552,
 14: 3.0254237288135593,
 15: 3.286206896551724,
 16: 3.0985550203779177,
 17: 2.90320765334834,
 18: 3.7843685879500093,
 19: 3.324675324675325,
 20: 3.146551724137931,
 21: 3.463302752293578,
 22: 2.2463054187192117,
 23: 3.55609756097561,
 24: 2.9939984996249063,
 25: 3.9701739850869924,
 26: 2.7937212079849854,
 27: 3.5274725274725274,
 28: 3.823254175890521,
 29: 3.598470363288719,
 30: 3.7618420274800908,
 31: 3.0542986425339365,
 32: 4.071736785329018,
 33: 4.168650217706822,
 34: 2.712962962962963,
 35: 3.1454112038140645,
 36: 3.020234291799787,
 37: 3.9893238434163703,
 38: 2.0511221945137157,
 39: 2.4361702127659575,
 40: 2.920570264765784,
 41: 1.6344086021505377,
 42: 2.328125,
 43

In [51]:
# f = open('average_ratings.json','w')
# f.write(str(average_ratings))
# f.close()
f = open('average_ratings.json','w')
json.dump(average_ratings,f)
f.close()

In [33]:
inverse = {rating: movie_id for movie_id,rating in average_ratings.items()}

In [66]:
best_movie_id = inverse[max(inverse)]
worst_movie_id = inverse[min(inverse)]

In [67]:
movie_titles[movie_titles['movie_id'] == worst_movie_id]

,movie_id,year,title
514,515,2005,Avia Vampire Hunter


In [65]:
movie_titles[movie_titles['movie_id'] == best_movie_id]

,movie_id,year,title
14960,14961,2003,Lord of the Rings: The Return of the King: Ext...


In [ ]:
## Worst and best movies check out :)